In [1]:
from flask import Flask, render_template, request, redirect, url_for
from flask import jsonify
import pandas as pd

df = pd.read_csv('Q&A.csv',header=0)

app = Flask(__name__)

In [2]:
@app.route('/')
def homes():
    return render_template('/index.html')

@app.route('/',methods = ['POST', 'GET'])
def home():
    return render_template('/index.html')

@app.route('/search',methods = ['POST', 'GET'])
def search():
    if request.method == 'POST':
        question = request.form['question']
        print(question)
        similar_q_id=''
        for i in range(len(df['questions'])):
            if df['questions'][i] == question:
                #answer=df['answers'][i]
                similar_q_id=df['similar_q_id'][i]
        
        if similar_q_id=='':
            return redirect(url_for('home'))
        else:
            return redirect(url_for('result',similar_q_id = similar_q_id))
    else:
        question = request.form['question']
        print(question)
        similar_q_id=''
        for i in range(len(df['questions'])):
            if df['questions'][i] == question:
                #answer=df['answers'][i]
                similar_q_id=df['similar_q_id'][i]
        
        if similar_q_id=='':
            return redirect(url_for('home'))
        else:
            return redirect(url_for('result',similar_q_id = similar_q_id))

@app.route('/result/<similar_q_id>')
def result(similar_q_id):
    my_value_count = df["similar_q_id"].value_counts()
    a=len(df[df['similar_q_id'] == int(similar_q_id)])
    b=df[df['similar_q_id'] == int(similar_q_id)]
    return render_template('/result.html',value = b)

@app.route('/answer',methods = ['POST', 'GET'])
def answer():
    if request.method == 'POST':
        question = request.form['question']
        print(question)
        answer=''
        for i in range(len(df['questions'])):
            if df['questions'][i] == question:
                answer=df['answers'][i]
        
        if answer=='':
            return redirect(url_for('home'))
        else:
            return redirect(url_for('success',answer = answer))
    else:
        question = request.form['question']
        print(question)
        answer=''
        for i in range(len(df['questions'])):
            if df['questions'][i] == question:
                answer=df['answers'][i]
        if answer=='':
            return redirect(url_for('home'))
        else:
            return redirect(url_for('success',answer = answer))

@app.route('/success/<answer>')
def success(answer):
    return render_template('answer.html', answer = answer)

In [ ]:
if __name__ == '__main__':
    app.run(host="0.0.0.0")

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://192.168.1.32:5000/ (Press CTRL+C to quit)
192.168.1.32 - - [19/Jul/2021 08:13:22] "GET / HTTP/1.1" 200 -
192.168.1.32 - - [19/Jul/2021 08:13:22] "GET /static/css/new.css HTTP/1.1" 200 -
192.168.1.32 - - [19/Jul/2021 08:13:22] "GET /static/css/new_1.css HTTP/1.1" 200 -
192.168.1.32 - - [19/Jul/2021 08:13:22] "GET /static/images/Searchs_130.png HTTP/1.1" 200 -
192.168.1.32 - - [19/Jul/2021 08:13:23] "GET /favicon.ico HTTP/1.1" 404 -
192.168.1.32 - - [19/Jul/2021 08:13:38] "POST /search HTTP/1.1" 302 -
192.168.1.32 - - [19/Jul/2021 08:13:38] "GET /result/3 HTTP/1.1" 200 -


What is Krsna consciousness?


192.168.1.32 - - [19/Jul/2021 08:13:38] "GET /static/css/new_1.css HTTP/1.1" 304 -
192.168.1.32 - - [19/Jul/2021 08:13:38] "GET /static/css/new.css HTTP/1.1" 200 -
192.168.1.32 - - [19/Jul/2021 08:13:38] "GET /static/images/Searchs_130.png HTTP/1.1" 200 -
192.168.1.32 - - [19/Jul/2021 08:13:51] "POST /answer HTTP/1.1" 302 -
192.168.1.32 - - [19/Jul/2021 08:13:51] "GET /success/Krishna%20consciousness%20is%20a%20state%20of%20awareness%20in%20which%20an%20individual%20acts%20in%20complete%20harmony%20with%20the%20Divine%20or%20the%20ultimate%20reality%20of%20Krishna.%20It%20is%20a%20form%20of%20Bhakti%20yoga%20%28or%20devotional%20service%29%20in%20which%20the%20purpose%20is%20to%20devote%20one%27s%20thoughts%2C%20actions%20and%20worship%20to%20pleasing%20Krishna%2C%20who%20some%20consider%20to%20be%20the%20supreme%20god.%20%20To%20act%20with%20Krishna%20consciousness%20is%20to%20free%20the%20self%20from%20the%20illusion%20that%20it%20is%20an%20individual%20body.%20It%20is%20a%20way%20to

What does Krishna Consciousness mean? 


192.168.1.32 - - [19/Jul/2021 08:13:51] "GET /static/css/new_1.css HTTP/1.1" 200 -
192.168.1.32 - - [19/Jul/2021 08:13:51] "GET /static/css/new.css HTTP/1.1" 200 -
192.168.1.32 - - [19/Jul/2021 08:13:51] "GET /static/images/Searchs_130.png HTTP/1.1" 200 -
192.168.1.32 - - [19/Jul/2021 08:13:51] "GET /favicon.ico HTTP/1.1" 404 -
192.168.1.32 - - [19/Jul/2021 08:14:03] "POST / HTTP/1.1" 200 -
192.168.1.32 - - [19/Jul/2021 08:14:03] "GET /static/css/new.css HTTP/1.1" 200 -
192.168.1.32 - - [19/Jul/2021 08:14:03] "GET /static/css/new_1.css HTTP/1.1" 200 -
192.168.1.32 - - [19/Jul/2021 08:14:04] "GET /static/images/Searchs_130.png HTTP/1.1" 200 -
192.168.1.32 - - [19/Jul/2021 08:14:04] "GET /favicon.ico HTTP/1.1" 404 -


In [114]:
@app.route('/')
def homes():
    return render_template('/index.html')

@app.route('/result')
def result():
    return render_template('/search_result.html')

@app.route('/',methods = ['POST', 'GET'])
def home():
    return render_template('/index.html')

@app.route('/success/<answer>')
def success(answer):
    return render_template('answer.html', answer = answer)

@app.route('/login',methods = ['POST', 'GET'])
def login():
    if request.method == 'POST':
        question = request.form['question']
        print(question)
        answer=''
        for i in range(len(df['questions'])):
            if df['questions'][i] == question:
                answer=df['answers'][i]
        
        if answer=='':
            return redirect(url_for('home'))
        else:
            print(question)
            return redirect(url_for('success',answer = answer))
    else:
        question = request.form['question']
        print(question)
        answer=''
        for i in range(len(df['questions'])):
            if df['questions'][i] == question:
                answer=df['answers'][i]
        if answer=='':
            return redirect(url_for('home'))
        else:
            return redirect(url_for('success',answer = answer))